In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

# Lista de pronombres en primera persona
first_person_pronouns = ["yo", "me", "mi", "mío", "mía", "mí", "nosotros", "nosotras", "nos", "nuestro", "nuestra", "nuestros", "nuestras"]

# Función para marcar presencia de pronombres de primera persona en un mensaje
def marcar_pronombres(message):
    tokens = word_tokenize(message.lower())
    presence = {pronoun: 1 if pronoun in tokens else 0 for pronoun in first_person_pronouns}
    return pd.Series(presence)

# Cargar datos
data = pd.read_csv ('linkCSV')


# Aplicar la función a cada mensaje
pronouns_presence = data['message'].apply(marcar_pronombres)

# Concatenar las nuevas columnas al DataFrame original
data = pd.concat([data, pronouns_presence], axis=1)

# Guardar el DataFrame actualizado en un nuevo archivo CSV
data.to_csv('linkCSV', index=False)

# Verificar que las nuevas columnas han sido añadidas correctamente
print(data.head())



In [ ]:
#intento en FreeLing

import pandas as pd
import subprocess
import os

# Lista de pronombres en primera persona
first_person_pronouns = ["yo", "me", "mi", "mío", "mía", "mí", "nosotros", "nosotras", "nos", "nuestro", "nuestra", "nuestros", "nuestras"]

# Función para usar FreeLing y contar pronombres de primera persona en un mensaje
def contar_pronombres_freeling(mensaje):
    # Guardar el mensaje en un archivo temporal
    with open("temp.txt", "w", encoding="utf-8") as file:
        file.write(mensaje)

    # Ejecutar FreeLing en el archivo temporal
    result = subprocess.run(['analyze', '-f', 'es.cfg'], input=mensaje, text=True, capture_output=True)

    # Leer el resultado
    output = result.stdout

    # Procesar el resultado para contar los pronombres
    tokens = [line.split()[0] for line in output.split('\n') if line and not line.startswith('#')]
    pronombres_encontrados = [palabra for palabra in tokens if palabra.lower() in first_person_pronouns]
    return len(pronombres_encontrados), ', '.join(pronombres_encontrados)

# Cargar datos
data = pd.read_csv('linkCSV')

# Aplicar la función a cada mensaje
data[['Total_pronombres', 'Pronombres_encontrados']] = data['message'].apply(lambda x: pd.Series(contar_pronombres_freeling(x)))

# Guardar el DataFrame actualizado en un nuevo archivo CSV
data.to_csv('linkCSV', index=False)

# Verificar que las nuevas columnas han sido añadidas correctamente
print(data.head())
